# Project: Running Llama 3.0 Locally

In [1]:
%%capture --no-stderr
%pip install -qU langchain langchain_community langchain-openai langchainhub langchain-ollama 

Commands to install Ollama on AWS Ubuntu ml.t3.xlarge.

In [ ]:
!sudo apt update && sudo apt upgrade --assume-yes
!sudo apt install curl --assume-yes
!curl --version
!curl -fsSL https://ollama.com/install.sh | sh
!ollama serve > server.out 2>&1 &
!ollama run llama3.1:8b > model.out 2>&1 &

In [20]:
!free -mh

               total        used        free      shared  buff/cache   available
Mem:            15Gi       6.6Gi       211Mi       0.0Ki       8.7Gi       8.5Gi
Swap:             0B          0B          0B


Below the client is created. 

In [13]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template="""You are an assistant that knows about Meta Llama. 
    
    If you don't know the answer, just say that you don't know. 
    
    Use one sentence to answer the question concisely.

    Question: {question} 

    Answer: 
    """,
    input_variables=["question"],
)
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
)
chain = prompt | llm | StrOutputParser()

Low lets test it with a question it shouldn't have information about. 

In [19]:
%%time
chain.invoke({'question':'What is the Knowledge cutoff for the text only version of Meta Llama 3?'})

CPU times: user 27 ms, sys: 0 ns, total: 27 ms
Wall time: 3.33 s


"I don't know."

This response looks correct as the knowledge cutoff for Llama 3.1 is December 2023.